In [2]:
import os
import time
import traceback
import sys
import pandas as pd


In [41]:
!pip install bert-score

Defaulting to user installation because normal site-packages is not writeable


In [42]:
from bert_score import score

In [46]:
P, R, F1 = score(["Bäckadräkten is Sweden's first unisex folk costume, designed in 2022 by musician Fredy Clue (pictured) and textile designer Ida Björs. Their mission was to encourage wider participation in folk traditions by ", "kingdom kingdom kingdom kingdom kingdom"], ["Duration should always be considered when a person is diagnosed with an adjustment disorder. Adjustment disorders are brief in nature. Symptom endorsement/count (either subjectively or objectively) is central, especially when considering a major depressive disorder or an adjustment disorder. Therefore, the correct answer is (C)", "apple apple apple apple apple"], verbose=True, rescale_with_baseline=True, lang="en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.10 seconds, 19.65 sentences/sec


In [47]:
F1

tensor([-0.1923,  0.1721])

In [45]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
sys.path.append('/nethome/cnguyen319/Counseling-QA/Johnny/')
from utils.openai_call import get_response
from utils.openai_setup import openai_setup

In [5]:
df = pd.read_csv('../results/outputs/Mistral-7B-Instruct-v0.3_few-shot-cot_None_1.csv')
df['raw_pred'] = [x.replace("\\n", "") for x in df['raw_pred']]
df['Explanation for correct answer'].fillna("", inplace=True)
df['Explanation for correct answer'] = [x.replace("Explanation: ", "") for x in df['Explanation for correct answer']]
df['raw_pred'] = [x.replace("Explanation: ", "") for x in df['raw_pred']]
df['raw_pred'] = [x.strip() for x in df['raw_pred']]
df['raw_pred'] = [x[:x.find("\n")] for x in df['raw_pred']]
df

/tmp/ipykernel_867978/2078642510.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Explanation for correct answer'].fillna("", inplace=True)


,Unnamed: 0,Source,Exam Name,Question #,Patient Demographic,Mental Status Exam,Presenting Problem,Question,Choice A,Choice B,...,Correct Answer,Explanation for correct answer,Other Contexts,Odd one out,Best answer,quote_rep,Answers,correct_answer_letter,raw_pred,parsed_pred
0,0,counselingexam.com,NCMHCE Narrative Exam 1,1,Name: Alex Clinical Issues: Hopelessness/depre...,Appearance: The client is dressed in a manner ...,First session You are a licensed mental health...,What would you discuss first with the client b...,Method of dealing with conflict,Substance use history,...,Suicidal ideation,When discussing the client's concerns and pres...,The client reports that his relationships with...,False,False,False,(A): Method of dealing with conflict\n(B): Sub...,D,The client's primary concern is his depression...,B
1,1,counselingexam.com,NCMHCE Narrative Exam 1,2,Name: Alex Clinical Issues: Hopelessness/depre...,Appearance: The client is dressed in a manner ...,First session You are a licensed mental health...,What is irrelevant to consider when gathering ...,Is he averse to taking medication for depression?,How willing is he to change his profession?,...,How willing is he to change his profession?,"He has already stated being ready to retire, a...",The client reports that his relationships with...,True,False,False,(A): Is he averse to taking medication for dep...,B,The client's preference for whiskey is not rel...,Z
2,2,counselingexam.com,NCMHCE Narrative Exam 1,3,Name: Alex Clinical Issues: Hopelessness/depre...,Appearance: The client is dressed in a manner ...,First session You are a licensed mental health...,"From a solution-focused therapy viewpoint, wha...",Ask the client to tell you about a time when h...,Ask the client to share what he has done in th...,...,"Ask the client the ""Miracle Question""",The “Miracle Question” is a technique used in ...,The client reports that his relationships with...,False,True,False,(A): Ask the client to tell you about a time w...,D,"The ""Miracle Question"" is a technique used in ...",D
3,3,counselingexam.com,NCMHCE Narrative Exam 1,4,Name: Alex Clinical Issues: Hopelessness/depre...,Appearance: The client is dressed in a manner ...,First session You are a licensed mental health...,What is the best way to build rapport with the...,Assess the client's current level of support,Use humor to lighten the mood,...,Express empathy and validate the client's feel...,Building rapport with a client is an important...,The client reports that his relationships with...,False,True,False,(A): Assess the client's current level of supp...,C,The best way to build rapport with the client ...,C
4,4,counselingexam.com,NCMHCE Narrative Exam 1,5,Name: Alex Clinical Issues: Hopelessness/depre...,Appearance: The client is dressed in a manner ...,First session You are a licensed mental health...,How would you approach the client's expressed ...,"""Do you think you won't have friends because d...","""Don't worry about your friends right now. Wha...",...,"""You're right to consider how big changes like...",This directly validates the client's emotions ...,The client reports that his relationships with...,False,False,True,"(A): ""Do you think you won't have friends beca...",D,The best approach would be to empathize with t...,D
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1616,1616,tests.com,NCMHCE Practice Exam,562,51 Initial Intake: Age: 26 S...,":\nMolly was initially guarded, but pleasant, ...",Molly came into individual counseling due to i...,Risk factors for generalized anxiety disorder ...,Genetic loading,Respiratory disturbance,...,Genetic loading,One third of the risk of developing general an...,NaN,False,False,False,(A): Genetic loading\n(B): Respiratory disturb...,A,The risk factors for generalized anxiety disor...,A
1617,1617,tests.com,NCMHCE Practice Exam,563,51 Initial Intake: Age: 26 S...,":\nMolly was initially guarded, but pleasant, ...",Molly came into individua

In [6]:
def load_prompt(prompt_file):
    prompt = ""
    with open(prompt_file, 'r') as f:
        prompt = f.read()
    return prompt

In [7]:
def prompt_type(question_data):
    case_1_string = f"Rationale 2 is more comprehensive, factual and coherent compared to Rationale 1"
    case_2_string = f"Rationale 2 is less comprehensive, factual and coherent compared to Rationale 1"
    case_3_string = f"Rationale 2 is equally comprehensive, factual and coherent compared to Rationale 1"
    output_structure_string = f"Your answer should first provide all reasoning followed by the chosen option string in the last line. The chosen option string should exactly match with one of the given options."
    prompt = load_prompt('../prompt_template/correctness.txt')
    prompt = prompt.format(question_data['demographic'], question_data['presenting_problem'], question_data['mental_exam'], question_data['context'], question_data['question'], question_data['answer'], question_data['true_explain'], question_data['pred_explain'], case_1_string, case_2_string, case_3_string, output_structure_string)
    return prompt


def get_eval(data_df):
    case_1_string = f"Rationale 2 is more comprehensive, factual and coherent compared to Rationale 1"
    case_2_string = f"Rationale 2 is less comprehensive, factual and coherent compared to Rationale 1"
    case_3_string = f"Rationale 2 is equally comprehensive, factual and coherent compared to Rationale 1"
    
    llm_answer_eval_list = []
    case_label_list = []

    
    option_string_list = [case_1_string, case_2_string, case_3_string]

    client = openai_setup()[0]
    for idx, row in enumerate(data_df.iterrows()):
        retry = True
        
        if idx%1 == 0:
            print("Index: ", idx)

        while retry:
            try:
                #select one column between answer and question_answered randomly
                message_list = [{"role": "system", "content": "You are a helpful, respectful, honest, and knowledgeable expert mental health counselor"}]
                messages = message_list.copy()
                prompt = prompt_type({'demographic': row[1]['Patient Demographic'], 'presenting_problem': row[1]['Presenting Problem'], 'mental_exam': row[1]['Mental Status Exam'], 'context': row[1]['Other Contexts'], 'question': row[1]['Question'], 'answer': row[1]['Answers'], 'true_explain': row[1]['Explanation for correct answer'], 'pred_explain': row[1]['raw_pred']})
                messages.append({'role': 'user', 'content': prompt})
                
                llm_response = get_response(client, messages, 'counselingqa-gpt4o', 0.1, 1024)
                case_label = ""
                
                if llm_response == "":
                    case_label = "No Response"
                
                else:
                
                    #check if any of the string present in the option_string_list is present in the response if yes then assign that string to case_label_English
                    # llm_response_check_list = (" ".join(llm_response.split("\n"))).split(".")
                    # get the last two sentences from the response
                    # llm_response_check = " ".join(llm_response_check_list[-3:])
                    llm_response_check = llm_response
                    
                    for option_string in option_string_list:
                        if option_string in llm_response_check:
                            case_label = option_string
                            break
                    
                    if case_label == "":
                        case_label = "No Response"
                
                time.sleep(0.3)

                # print("LLM Response: ", llm_response)
                print("Case Label: ", case_label)

                llm_answer_eval_list.append(llm_response)
                case_label_list.append(case_label)
                #code for saving intermediate files

                retry = False

            except Exception as e:
                print("Error at index: ", idx)
                traceback.print_exc()
                print("Error: ", e)
                #check if the error contains the substring Request timed out: HTTPSConnectionPool or rate limit

                if "Request timed out: HTTPSConnectionPool" in str(e) or "rate limit" in str(e) or "timed out" or "No Response" in str(e):
                    print("Sleeping for 10 seconds")
                    time.sleep(10)
                    continue

                else:
                    #check if llm_response_English exists and is not empty
                    if llm_response:
                        llm_answer_eval_list.append(llm_response)
                        case_label_list.append(case_label)
                    else:
                        if "This model's maximum context length is 8192 tokens" in str(e):
                            llm_answer_eval_list.append("Max Context Length Exceeded")
                            case_label_list.append("Max Context Length Exceeded")

                        else:
                            llm_answer_eval_list.append(str(e))
                            case_label_list.append(str(e))

                    # print("LLM Response: ", llm_response)
                    print("Case Label: ", case_label)
                    retry = False
                    continue



    data_df["llm_answer_eval"] = llm_answer_eval_list
    data_df["case_label"] = case_label_list
        
    return data_df

In [8]:
new_df = get_eval(df)

Index:  0
Case Label:  Rationale 2 is less comprehensive, factual and coherent compared to Rationale 1
Index:  1
Case Label:  No Response
Index:  2
Case Label:  Rationale 2 is more comprehensive, factual and coherent compared to Rationale 1
Index:  3
Case Label:  Rationale 2 is more comprehensive, factual and coherent compared to Rationale 1
Index:  4
Case Label:  Rationale 2 is more comprehensive, factual and coherent compared to Rationale 1
Index:  5
Case Label:  Rationale 2 is more comprehensive, factual and coherent compared to Rationale 1
Index:  6
Case Label:  Rationale 2 is more comprehensive, factual and coherent compared to Rationale 1
Index:  7
Case Label:  Rationale 2 is more comprehensive, factual and coherent compared to Rationale 1
Index:  8
Case Label:  Rationale 2 is more comprehensive, factual and coherent compared to Rationale 1
Index:  9
Case Label:  Rationale 2 is more comprehensive, factual and coherent compared to Rationale 1
Index:  10
Case Label:  Rationale 2 is

In [9]:
new_df.to_csv('output.csv')

In [13]:
from collections import Counter
Counter(new_df['case_label'])

Counter({'Rationale 2 is less comprehensive, factual and coherent compared to Rationale 1': 906,
         'Rationale 2 is more comprehensive, factual and coherent compared to Rationale 1': 682,
         'Rationale 2 is equally comprehensive, factual and coherent compared to Rationale 1': 29,
         'No Response': 4})